---
# Trabalho 1 - Tópicos Especiais em Matemática Aplicada

**Alunos/Matricula:** João V. Farias & Renan V. Guedes / 221022604 & 221031363

**Arquitetura Usada:** Encoder-Decoder

**Dataset Link:** V1: [D-Talk](https://www.tensorflow.org/datasets/catalog/ted_hrlr_translate#ted_hrlr_translatefr_to_pt) from TensorFlow.Datasets

---

### **Projeto para traduzir mensagens do Francês para o Portugues**  

Neste projeto, vamos explorar e comparar três arquiteturas de redes neurais para tradução automática do francês para o português, usando o *dataset* TED Talks do *Open Translation Project*. A ideia é testar modelos do tipo **Encoder-Decoder**, analisando suas diferenças e impacto na qualidade da tradução.  

Os três modelos que vamos treinar são:  

1. **LSTM (Long Short-Term Memory)**  
   - Um modelo básico de rede recorrente bidirecional. O **Encoder** processa a frase em francês e gera um contexto, enquanto o **Decoder** usa esse contexto para formar a tradução em português.  
   - A principal vantagem desse modelo é sua capacidade de lidar com dependências de longo prazo nas sequências.  

2. **LSTM com Mecanismos de Atenção**  
   - Uma versão aprimorada do modelo anterior, adicionando uma camada de atenção (no nosso caso, escolhemos Luong).
   - A atenção ajuda o modelo a "olhar" para partes específicas da frase de entrada enquanto traduz, melhorando a coerência e precisão.  

3. **Transformers**  
   - Uma abordagem mais moderna, baseada em **autoatenção**, tirando o uso uso de redes recorrentes.  
   - Trabalha com processamento paralelo, usando *Multi-Head Attention* e *Positional Encoding* para entender relações entre palavras, mesmo quando estão distantes na frase.  

**Como vamos testar os modelos?**  
- **Dataset**: Vamos usar cerca de 52.000 pares de frases (francês-português) para treinar, além de 1.200 para validação e 1.800 para teste.  
- **Pré-processamento**: Faremos a tokenização com *SubwordTextEncoder* para reduzir palavras fora do vocabulário (*out-of-vocabulary* – OOV).  
- **Treinamento**: Otimização com Adam, acompanhando o loss e a acurácia durante o processo.  
- **Avaliação**: Vamos comparar os resultados usando a métrica BLEU e analisar exemplos práticos das traduções.  

**O que esperamos encontrar?**  
- entender qual desses modelos tem o melhor equilíbrio entre qualidade de tradução e eficiência computacional.
- provável que o transformer tenha um desempenho superior, já que conseguem processar frases de forma mais eficiente, enquanto o modelo com LSTM e atenção deve mostrar um avanço significativo sobre a versão básica de LSTM.

No fim das contas, tentamos compreender melhor como essas diferentes técnicas vão trazer como resultado.

---

## 📚 Importando as bibliotecas necessárias

In [23]:
# Primeiro, vamos importar todas as bibliotecas que vamos precisar ao longo do Projeto
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os

# Para processamento de texto
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Componentes do Keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Attention, MultiHeadAttention, Concatenate, TimeDistributed
from tensorflow.keras.optimizers import Adam

# Para visualização dos resultados
from sklearn.metrics import confusion_matrix

tf.config.optimizer.set_jit(True)  # Ativa o XLA JIT compilation
tf.keras.mixed_precision.set_global_policy('mixed_float16') # Força o TensorFlow a usar precisão mista

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.18.0


## 🔍 Carregando e Preparando os Dados

In [24]:
examples, metadata = tfds.load('ted_hrlr_translate/fr_to_pt', with_info=True, as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']


 Dataset carregado e preprocessado com sucesso! 🎉


A gente começou limpando os textos, retirando aqueles símbolos tipo ?, !, . e até o ¿, que no francês e no português não são usados do mesmo jeito. Esses sinais também atrapalham na hora de treinar, então decidimos removê-los pra deixar tudo mais uniforme e fácil de lidar..

In [ ]:
def preprocess_text(text):
    text = tf.strings.regex_replace(text, r"([?.!,¿])", r" \1 ")
    text = tf.strings.regex_replace(text, r'[" "]+', " ")
    text = tf.strings.strip(text)
    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text

A ideia aqui foi organizar os dados de forma que a gente conseguisse trabalhar numa boa com eles. Separamos as frases e garantimos que cada exemplo estivesse no formato certo. Foi interessante ver como uma boa preparação dos dados já facilita bastante o treino dos modelos depois.

In [ ]:
def prepare_dataset(examples, max_samples=None):
    fr_texts, pt_texts = [], []
    for fr, pt in examples:
        fr_texts.append(preprocess_text(fr).numpy().decode('utf-8'))
        pt_texts.append(preprocess_text(pt).numpy().decode('utf-8'))
        if max_samples and len(fr_texts) >= max_samples:
            break
    return fr_texts, pt_texts

Pra não sobrecarregar o sistema e também evitar overfitting, a gente pegou 50 mil amostras pra treinamento e 10 mil pra validação.

In [ ]:
fr_texts, pt_texts = prepare_dataset(train_examples, max_samples=50000)

In [ ]:
val_fr_texts, val_pt_texts = prepare_dataset(val_examples, max_samples=10000)

## 🛠️ Configurando os Tokenizers

Criamos uma função que faz a tokenização e o padding das frases. Usamos o token `[OOV]` pra lidar com palavras que não estão no vocabulário e, depois, aplicamos padding pra deixar todas as sequências com o mesmo tamanho. Assim, fica mais fácil converter o texto em sequências numéricas e alimentar a rede neural sem complicação.

In [27]:
def tokenize_and_pad(fr_texts, pt_texts, max_input_length, max_target_length):
    tokenizer_fr = Tokenizer(filters='', oov_token='[OOV]')
    tokenizer_fr.fit_on_texts(fr_texts)
    input_vocab_size = len(tokenizer_fr.word_index) + 1

    tokenizer_pt = Tokenizer(filters='', oov_token='[OOV]')
    tokenizer_pt.fit_on_texts(pt_texts)
    target_vocab_size = len(tokenizer_pt.word_index) + 1

    # Convert texts to sequences
    fr_sequences = tokenizer_fr.texts_to_sequences(fr_texts)
    pt_sequences = tokenizer_pt.texts_to_sequences(pt_texts)

    # Prepare decoder inputs and outputs
    decoder_inputs = [seq[:-1] for seq in pt_sequences]
    decoder_outputs = [seq[1:] for seq in pt_sequences]

    # Pad sequences
    encoder_inputs = pad_sequences(fr_sequences, maxlen=max_input_length, padding='post')
    decoder_inputs = pad_sequences(decoder_inputs, maxlen=(max_target_length - 1), padding='post')
    decoder_outputs = pad_sequences(decoder_outputs, maxlen=(max_target_length - 1), padding='post')

    return encoder_inputs, decoder_inputs, decoder_outputs, input_vocab_size, target_vocab_size


In [25]:
max_input_length = 50
max_target_length = 50

In [28]:
train_encoder_inputs, train_decoder_inputs, train_decoder_outputs, input_vocab_size, target_vocab_size = tokenize_and_pad(
    fr_texts, pt_texts, max_input_length, max_target_length
)

In [ ]:
val_encoder_inputs, val_decoder_inputs, val_decoder_outputs, _, _ = tokenize_and_pad(
    val_fr_texts, val_pt_texts, max_input_length, max_target_length
)

# 🤖 Modelo 1: LSTM Básico

No modelo LSTM, definimos a função build_lstm_model que basicamente cria um encoder e um decoder usando teacher forcing. A ideia é que o encoder pegue a sequência de entrada e o decoder aprenda a gerar a saída com base nas informações que recebeu, sempre “corrigindo” o que já foi gerado. As camadas de inferência, que seriam usadas na hora de testar ou gerar traduções de verdade, a gente deixou de lado, mas o ChatGPT e o Deepseek recomendaram para gente, mas não era o foco do trabalho.

In [29]:
def build_lstm_model(input_vocab_size, target_vocab_size,
                     max_input_len, max_target_len,
                     latent_units=256, embedding_dim=256):
    # Encoder
    encoder_inputs = Input(shape=(max_input_len,), name="encoder_inputs")
    encoder_embed = Embedding(input_vocab_size, embedding_dim, name="encoder_embedding")(encoder_inputs)
    encoder_lstm = LSTM(latent_units, return_state=True, name="encoder_lstm")
    _, state_h, state_c = encoder_lstm(encoder_embed)

    # Decoder
    decoder_inputs = Input(shape=(max_target_len-1,), name="decoder_inputs")
    decoder_embed = Embedding(target_vocab_size, embedding_dim, name="decoder_embedding")(decoder_inputs)
    decoder_lstm = LSTM(latent_units, return_sequences=True, name="decoder_lstm")
    decoder_outputs = decoder_lstm(decoder_embed, initial_state=[state_h, state_c])
    decoder_dense = Dense(target_vocab_size, activation='softmax', dtype='float32', name="decoder_dense")
    decoder_outputs = decoder_dense(decoder_outputs)


    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.encoder_inputs = encoder_inputs
    model.encoder_embedding = model.get_layer("encoder_embedding")
    model.encoder_lstm = model.get_layer("encoder_lstm")
    model.decoder_embedding = model.get_layer("decoder_embedding")
    model.decoder_lstm = model.get_layer("decoder_lstm")
    model.decoder_dense = decoder_dense
    return model


## Criando e Compilando os modelos

Para todos os nossos modelos, a configuração foi padrão: usamos o otimizador Adam com learning rate de 1e-4 e clipnorm de 1.0, a loss foi definida como 'sparse_categorical_crossentropy' e a métrica de acurácia. Essa padronização ajudou a gente a comparar os resultados dos modelos sem ter que ficar ajustando vários parâmetros diferentes.

In [31]:
lstm_model = build_lstm_model(
    input_vocab_size=len(fr_tokenizer.word_index)+1,
    target_vocab_size=len(pt_tokenizer.word_index)+1,
    max_input_len=max_fr,
    max_target_len=max_pt
)

lstm_model.compile(
    optimizer=Adam(learning_rate=1e-4, clipnorm=1.0),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

lstm_model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, 151)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, 154)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_embedding         │ (None, 151, 256)       │      9,464,576 │ encoder_inputs[0][0]   │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_embedding         │ (None, 154, 256)       │      9,603,328 │ decoder_inputs[0][0]   │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ encoder_lstm (LSTM)       │ [(None, 256), (None,   │        525,312 │ encoder_embedding[0][… │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_lstm (LSTM)       │ (None, 154, 256)       │        525,312 │ decoder_embedding[0][… │
│                           │                        │                │ encoder_lstm[0][1],    │
│                           │                        │                │ encoder_lstm[0][2]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast_2 (Cast)             │ (None, 154, 256)       │              0 │ decoder_lstm[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_dense (Dense)     │ (None, 154, 37513)     │      9,640,841 │ cast_2[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 29,759,369 (113.52 MB)

 Trainable params: 29,759,369 (113.52 MB)

 Non-trainable params: 0 (0.00 B)

## Treinando o Modelo

In [33]:
# Treinamento
history = lstm_model.fit(
    [train_encoder_inputs, train_decoder_inputs], train_decoder_outputs,
    validation_data=([val_encoder_inputs, val_decoder_inputs], val_decoder_outputs),
    epochs=5,
    batch_size=10,
    validation_split=0.2,
    verbose=1
)

Epoch 1/5
4388/4388 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8836 - loss: 1.8984

ValueError: Layer "functional_2" expects 2 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=(None, 109) dtype=int32>]

# 🤖 Modelo 2: LSTM (Luong)

A gente adaptou o modelo LSTM para incluir um mecanismo de atenção – escolhemos o esquema de Luong mesmo, pois pareceu mais simples e direto. Aqui, em vez de usar teacher forcing, o decoder “presta atenção” nas partes relevantes do encoder durante a geração da tradução. Foi interessante notar como a atenção ajudou a melhorar o alinhamento entre as palavras de entrada e saída, mesmo com uma estrutura parecida com a LSTM simples.


In [35]:
def build_lstm_attention_model(input_vocab_size, target_vocab_size, max_input_len, max_target_len, latent_units=256, embedding_dim=256):
    # Encoder
    encoder_inputs = Input(shape=(max_input_len,))
    encoder_embed = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm = LSTM(latent_units, return_sequences=True, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embed)

    # Decoder
    decoder_inputs = Input(shape=(max_target_len - 1,))
    decoder_embed = Embedding(target_vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(latent_units, return_sequences=True)
    decoder_outputs = decoder_lstm(decoder_embed, initial_state=[state_h, state_c])

    # Attention
    attention = Attention()([decoder_outputs, encoder_outputs])
    decoder_concat = tf.concat([decoder_outputs, attention], axis=-1)
    decoder_dense = Dense(target_vocab_size, activation='softmax', dtype='float32')
    decoder_outputs = decoder_dense(decoder_concat)

    return Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [36]:
lstm_attn_model = build_lstm_attention_model(
    input_vocab_size=len(fr_tokenizer.word_index)+1,
    target_vocab_size=len(pt_tokenizer.word_index)+1,
    max_input_len=max_fr,
    max_target_len=max_pt
)

# optimizer=Adam(learning_rate=0.001)
lstm_attn_model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

lstm_attn_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:1387: UserWarning: Layer 'luong_attention' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Dimensions must be equal, but are 154 and 151 for '{{node add}} = AddV2[T=DT_HALF](ExpandDims, Placeholder_1)' with input shapes: [?,1,154,256], [?,151,256].''
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'luong_attentio

ValueError: Exception encountered when calling LuongAttention.call().

[1mCould not automatically infer the output shape / dtype of 'luong_attention' (of type LuongAttention). Either the `LuongAttention.call()` method is incorrect, or you need to implement the `LuongAttention.compute_output_spec() / compute_output_shape()` method. Error encountered:

Dimensions must be equal, but are 154 and 151 for '{{node add}} = AddV2[T=DT_HALF](ExpandDims, Placeholder_1)' with input shapes: [?,1,154,256], [?,151,256].[0m

Arguments received by LuongAttention.call():
  • args=('<KerasTensor shape=(None, 154, 256), dtype=float16, sparse=False, name=keras_tensor_35>', '<KerasTensor shape=(None, 151, 256), dtype=float16, sparse=False, name=keras_tensor_30>')
  • kwargs=<class 'inspect._empty'>

In [ ]:
lstm_attn_model.fit(
    [train_encoder_inputs, train_decoder_inputs], train_decoder_outputs,
    validation_data=([val_encoder_inputs, val_decoder_inputs], val_decoder_outputs),
    epochs=5,
    batch_size=10,
    validation_split=0.2,
    verbose=1
)

# 🤖 Modelo 3: Transformador

Por fim, implementamos um Transformer, que é uma abordagem totalmente diferente, sem recorrência. No código, o encoder e o decoder são feitos de várias camadas com MultiHeadAttention, normalização e dropout. Cada embedding é escalado e normalizado antes de entrar nas camadas, e no final temos uma camada densa com softmax pra prever as palavras. Esse modelo chamou nossa atenção porque, apesar de mais complexo, mostrou um potencial único pra capturar relações mais profundas nas sentenças como o professor tinha dito em sala.

In [ ]:
def build_transformer_model(input_vocab_size, target_vocab_size, max_input_len, max_target_len, num_layers=2, d_model=256, num_heads=8, dff=512, dropout_rate=0.1):
    # Encoder
    encoder_inputs = Input(shape=(max_input_len,))
    encoder_embed = Embedding(input_vocab_size, d_model)(encoder_inputs)
    encoder_embed *= tf.math.sqrt(tf.cast(d_model, tf.float32))  # Scale embeddings
    encoder_embed = LayerNormalization(epsilon=1e-6)(encoder_embed)

    for _ in range(num_layers):
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(encoder_embed, encoder_embed)
        attn_output = Dropout(dropout_rate)(attn_output)
        encoder_embed = LayerNormalization(epsilon=1e-6)(encoder_embed + attn_output)

        ffn_output = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])(encoder_embed)
        ffn_output = Dropout(dropout_rate)(ffn_output)
        encoder_embed = LayerNormalization(epsilon=1e-6)(encoder_embed + ffn_output)

    # Decoder
    decoder_inputs = Input(shape=(max_target_len - 1,))
    decoder_embed = Embedding(target_vocab_size, d_model)(decoder_inputs)
    decoder_embed *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    decoder_embed = LayerNormalization(epsilon=1e-6)(decoder_embed)

    for _ in range(num_layers):
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(decoder_embed, decoder_embed)
        attn_output = Dropout(dropout_rate)(attn_output)
        decoder_embed = LayerNormalization(epsilon=1e-6)(decoder_embed + attn_output)

        ffn_output = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])(decoder_embed)
        ffn_output = Dropout(dropout_rate)(ffn_output)
        decoder_embed = LayerNormalization(epsilon=1e-6)(decoder_embed + ffn_output)

    # Final output
    decoder_outputs = Dense(target_vocab_size, activation='softmax', dtype='float32')(decoder_embed)
    return Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
transformer_model = build_transformer_model(input_vocab_size, target_vocab_size, max_input_length, max_target_length)

# optimizer=Adam(learning_rate=0.001)
transformer_model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

transformer_model.summary()

# 📊 Visualizando os Resultados

In [ ]:
def visualize_results(history):
    plt.figure(figsize=(12, 4))

    # Plot da loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Treino')
    plt.plot(history.history['val_loss'], label='Validação')
    plt.title('Loss ao Longo do Treinamento')
    plt.xlabel('Época')
    plt.ylabel('Loss')
    plt.legend()

    # Plot da acurácia
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Treino')
    plt.plot(history.history['val_accuracy'], label='Validação')
    plt.title('Acurácia ao Longo do Treinamento')
    plt.xlabel('Época')
    plt.ylabel('Acurácia')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
def plot_accuracy_by_position(y_true, y_pred):
    """
    Plots the token-level accuracy at each sequence position.

    Parameters:
      - y_true: numpy array of shape (num_samples, sequence_length)
                containing the ground truth token indices.
      - y_pred: numpy array of shape (num_samples, sequence_length)
                containing the predicted token indices.
    """
    seq_length = y_true.shape[1]
    accuracies = []
    for pos in range(seq_length):
        # Compute the fraction of tokens predicted correctly at this position.
        pos_accuracy = np.mean(y_true[:, pos] == y_pred[:, pos])
        accuracies.append(pos_accuracy)

    plt.figure(figsize=(10, 6))
    plt.plot(range(seq_length), accuracies, marker='o')
    plt.xlabel("Token Position")
    plt.ylabel("Accuracy")
    plt.title("Accuracy by Token Position")
    plt.ylim(0, 1)
    plt.grid(True)
    plt.show()


In [ ]:
def plot_length_distribution(y_true, y_pred):
    # Calculando comprimentos (ignorando padding)
    true_lengths = [len([x for x in seq if x != 0]) for seq in y_true]
    pred_lengths = [len([x for x in seq if x != 0]) for seq in y_pred]

    plt.figure(figsize=(12, 6))
    plt.hist([true_lengths, pred_lengths], label=['Real', 'Previsto'],
             alpha=0.7, bins=20)
    plt.title('Distribuição do Comprimento das Traduções')
    plt.xlabel('Comprimento da Sequência')
    plt.ylabel('Frequência')
    plt.legend()
    plt.show()

In [ ]:
def plot_confusion_heatmap(y_true, y_pred, token_labels):
    """
    Plots a confusion matrix (as a heatmap) for token predictions.

    Parameters:
      - y_true: numpy array of token indices (can be 2D or flattened)
      - y_pred: numpy array of token indices (can be 2D or flattened)
      - token_labels: list of strings that maps each token index to a label.
                      For example: ["PAD", "[START]", "[END]", "bonjour", ...]
    """
    # If the inputs are 2D (num_samples x seq_length), flatten them.
    if y_true.ndim > 1:
        y_true = y_true.flatten()
    if y_pred.ndim > 1:
        y_pred = y_pred.flatten()

    # Compute the confusion matrix.
    labels = np.arange(len(token_labels))
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=token_labels,
                yticklabels=token_labels)
    plt.xlabel("Predicted Token")
    plt.ylabel("True Token")
    plt.title("Confusion Matrix of Token Predictions")
    plt.show()

In [ ]:
# Function to generate predictions
def generate_predictions(model, encoder_inputs, decoder_inputs, max_target_length):
    predictions = model.predict([encoder_inputs, decoder_inputs])
    predictions = np.argmax(predictions, axis=-1)
    return predictions

# Token labels for confusion matrix (excluding padding token 0)
token_labels = list(range(1, target_vocab_size))

models = {
    "LSTM Básico": lstm_model,
    "LSTM (Luong)": lstm_attn_model,
    "Transformer": transformer_model
}


# Analyze each model
for name, model in models.items():
    print(f"\nAnalyzing {name} model...")

    # Generate predictions
    train_predictions = generate_predictions(model, train_encoder_inputs, train_decoder_inputs, max_target_length)
    val_predictions = generate_predictions(model, val_encoder_inputs, val_decoder_inputs, max_target_length)

    # Visualize training results
    visualize_results(history)

    # Plot accuracy by position
    plot_accuracy_by_position(train_decoder_outputs, train_predictions)

    # Plot sequence length distribution
    plot_length_distribution(train_decoder_outputs, train_predictions)

    # Plot confusion matrix heatmap
    plot_confusion_heatmap(train_decoder_outputs, train_predictions, token_labels)